In [2]:
%matplotlib inline
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
from scipy.stats import poisson, kurtosis
import matplotlib.patches as mpatches
from functools import partial
import pickle
import os
import demixing as dm
from demixing import MLP, HiddenLayer

nneuron = 61
min_angle = -90
max_angle = 90
sprefs = np.linspace(min_angle, max_angle, nneuron)
eps = np.finfo(np.float64).eps
sigtc_sq = float(10**2)

In [ ]:
pkl_file = open('readout.pkl', 'rb')
posts, testsets = pickle.load(pkl_file)

In [4]:
posts = {}
testsets = {}
for s_i in range(31):
    file_name = 'Minicon_Outputs/output_post_2/post_2_' + str(s_i) + '.pkl'
    if os.path.isfile(file_name):
        pkl_file = open(file_name, 'rb')
        p, r, c, delta_s = pickle.load(pkl_file)
        posts[delta_s] = p
        testsets[delta_s] = r

In [14]:
nns = {}
for i in range(1, 401):
    file_name = 'Minicon_Outputs/output_nn_tests_2/nn_tests_2_' + str(i) + '.pkl'
    if os.path.isfile(file_name):
        pkl_file = open(file_name, 'rb')
        nn, nnx, valid_mse, _, _ = pickle.load(pkl_file)
        nns[i] = (nn, nnx, valid_mse)

In [12]:
for i in range(1, len(nns) + 1):
    print i
    nn, nnx, valid_mse = nns[i]
    x = []
    y = []
    for s_i in range(31):
        x.append(dm.get_hu_responses(testsets[s_i], nn))
        y.append(np.array((1/posts[s_i]['var_s1'], 1/posts[s_i]['var_s2'])))
    y = np.concatenate(y, axis=1).T
    x = np.concatenate(x)
    inds = range(len(x))
    np.random.shuffle(inds)
    x_shuf = x[inds]
    y_shuf = y[inds]
    validset = x_shuf[0:2000], y_shuf[0:2000]
    trainset = x_shuf[2000:], y_shuf[2000:]
    
    hus, vpost = validset
    weights = np.linalg.lstsq(trainset[0], trainset[1])[0]
    lin_preds = np.dot(hus, weights)
    sum_preds = np.sum(hus, axis=1)
    kurt_preds = kurtosis(hus, axis=1)
    vp = np.concatenate((vpost.T[0], vpost.T[1]))
    lin_preds = np.concatenate((lin_preds.T[0], lin_preds.T[1]))
    sum_preds = np.concatenate((sum_preds, sum_preds))
    kurt_preds = np.concatenate((kurt_preds, kurt_preds))

    lin_corr = np.corrcoef(vp, lin_preds)
    sum_corr = np.corrcoef(vp, sum_preds)
    kurt_corr = np.corrcoef(vp, kurt_preds)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


KeyboardInterrupt: 

In [17]:
lin_corrs1 = np.zeros(400)
lin_corrs2 = np.zeros(400)
for i in range(1, 400):
    print i
    nn, nnx, valid_mse = nns[i]
    x1 = []
    x2 = []
    y = []
    for s_i in range(31):
        x1.append(dm.get_hu_responses(testsets[s_i], nn))
        x2.append(testsets[s_i])
        y.append(np.array((1/posts[s_i]['var_s1'], 1/posts[s_i]['var_s2'])))
    y = np.concatenate(y, axis=1).T
    x1 = np.concatenate(x1)
    x2 = np.concatenate(x2)
    inds = range(len(x))
    np.random.shuffle(inds)
    x1_shuf = x1[inds]
    x2_shuf = x2[inds]
    y_shuf = y[inds]
    validset1 = x1_shuf[0:2000], y_shuf[0:2000]
    trainset1 = x1_shuf[2000:], y_shuf[2000:]
    validset2 = x2_shuf[0:2000], y_shuf[0:2000]
    trainset2 = x2_shuf[2000:], y_shuf[2000:]
    
    hus, vpost = validset1
    inputs, vpost = validset2
    weights1 = np.linalg.lstsq(trainset1[0], trainset1[1])[0]
    weights2 = np.linalg.lstsq(trainset2[0], trainset2[1])[0]
    lin_preds1 = np.dot(hus, weights1)
    lin_preds2 = np.dot(inputs, weights2)
    vp = np.concatenate((vpost.T[0], vpost.T[1]))
    lin_preds1 = np.concatenate((lin_preds1.T[0], lin_preds1.T[1]))
    lin_preds2 = np.concatenate((lin_preds2.T[0], lin_preds2.T[1]))

    lin_corrs1[i] = np.corrcoef(vp, lin_preds1)[0, 1]
    lin_corrs2[i] = np.corrcoef(vp, lin_preds2)[0, 1]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [21]:
lin_corrs1 - lin_corrs2

array([ 0.        ,  0.08064477,  0.04642026,  0.04078216,  0.07385635,
        0.08944091,  0.08466064,  0.05494786,  0.05084443,  0.11200913,
        0.04259298,  0.06821382,  0.06621984,  0.0425205 ,  0.06561545,
        0.09523969,  0.0796194 ,  0.10909944,  0.06964411,  0.03334399,
        0.03229243,  0.0610537 ,  0.05947424,  0.10408833,  0.09555264,
       -0.13665291,  0.07265146,  0.08272077,  0.05719109,  0.06377665,
        0.06439674,  0.0613034 ,  0.07974163,  0.01933558,  0.08435245,
        0.06135039,  0.06970729,  0.06499436,  0.0261104 ,  0.12018273,
        0.0762013 ,  0.09190541,  0.07371412,  0.08496458,  0.04205841,
        0.07018721,  0.08384011,  0.09100609,  0.04171498,  0.04519712,
        0.08206115,  0.08842851,  0.0788248 ,  0.087195  ,  0.04804993,
        0.11170285,  0.07056841,  0.04819994,  0.05821761,  0.07613236,
        0.07157269,  0.08376575,  0.06644313,  0.08216004,  0.09091233,
        0.0897165 ,  0.07838503,  0.04199557,  0.09834651,  0.06

In [13]:
weights

array([[  1.64005245e-19,   9.97246685e-20],
       [ -1.11369247e-15,   1.77809156e-16],
       [ -4.17200996e-16,   3.21791205e-16],
       [  1.21690852e-15,  -3.39138440e-16],
       [ -4.33072391e-02,   6.58590112e-02],
       [ -3.01055902e-02,   4.04471625e-02],
       [ -3.84831197e-02,   2.56913512e-02],
       [ -3.63317652e-02,   2.30856906e-02],
       [ -2.94757965e-02,   1.51337488e-02],
       [ -2.52071446e-02,   1.31570506e-02],
       [ -2.05754151e-02,   9.20077572e-03],
       [ -1.55305014e-02,   7.22850741e-03],
       [ -1.13091431e-02,   5.99656578e-03],
       [ -7.15255586e-03,   4.20176860e-03],
       [ -3.12904442e-03,   2.78464476e-03],
       [  3.15031399e-04,   2.02087493e-03],
       [  3.12635764e-03,   1.46338117e-03],
       [  4.94676884e-03,   1.32631603e-03],
       [  5.91283892e-03,   1.23112852e-03],
       [  6.05526689e-03,   1.36923955e-03],
       [  5.56757622e-03,   1.44112319e-03],
       [  4.15624185e-03,   1.66014005e-03],
       [  